# Waste Transfer Learning Assignment

Full ready-to-run notebook for the Coursera peer-graded assignment: **Classify Waste Products Using Transfer Learning**.

**How to use**:
- Update the dataset paths (`train_dir`, `val_dir`, `test_dir`) to point to your data folders (each folder should contain class subfolders).
- Run the cells in order (recommended in Google Colab or Jupyter).
- The notebook prints and saves the outputs required by Coursera for screenshot submission.

---


In [ ]:
# Imports and configuration
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ---------------------------
# User configuration - EDIT
# ---------------------------
# Point these to your dataset directory structure (train/val/test each with class subfolders)
train_dir = '/content/data/train'    # <-- update
val_dir   = '/content/data/val'      # <-- update
test_dir  = '/content/data/test'     # <-- update

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_EPOCHS_EXTRACT = 8
NUM_EPOCHS_FINETUNE = 6
SEED = 123
INDEX_TO_PLOT = 1  # as required by the assignment

# ---------------------------
# Task 1: Print TF version
# ---------------------------
print("TensorFlow version:", tf.__version__)

# ---------------------------
# Reproducibility
# ---------------------------
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# ---------------------------
# Data generators
# ---------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# test_datagen - requested in tasks
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

# ---------------------------
# Task 2: create test_generator using test_datagen
# ---------------------------
# Important: shuffle=False so that predictions correspond to file order

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# For screenshot: show an example batch from test_generator
imgs, labels = next(test_generator)
print('Example test batch shape (images, labels):', imgs.shape, labels.shape)

# ---------------------------
# Task 3: Print the length of train_generator
# ---------------------------
print('Length of train_generator (number of batches):', len(train_generator))

# ---------------------------
# Build the model - Extract Features Model
# ---------------------------
num_classes = len(train_generator.class_indices)
print('Number of classes:', num_classes)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Extract-features stage: freeze base

inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

extract_model = keras.Model(inputs, outputs, name='extract_features_model')

# ---------------------------
# Task 4: Print the model summary
# ---------------------------
print('\\n--- Extract Features Model Summary ---')
extract_model.summary()

# ---------------------------
# Task 5: Compile the model
# ---------------------------
extract_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print('\\nModel compiled (extract features).')

# ---------------------------
# Train Extract-Features Model
# ---------------------------
history_extract = extract_model.fit(
    train_generator,
    epochs=NUM_EPOCHS_EXTRACT,
    validation_data=validation_generator
)

# Save history plot for Task 6 (accuracy curves for extract-features model)
plt.figure()
plt.plot(history_extract.history['accuracy'], label='train_accuracy')
plt.plot(history_extract.history['val_accuracy'], label='val_accuracy')
plt.title('Extract-Features Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('task6_extract_accuracy.png', bbox_inches='tight')
print("Saved 'task6_extract_accuracy.png' (Task 6).")

# ---------------------------
# Fine-tuning stage
# ---------------------------
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.8)  # unfreeze top 20%
for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= fine_tune_at

fine_tune_inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(fine_tune_inputs)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
fine_tune_outputs = layers.Dense(num_classes, activation='softmax')(x)

fine_tune_model = keras.Model(fine_tune_inputs, fine_tune_outputs, name='fine_tune_model')

fine_tune_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print('\\n--- Fine-tune Model Summary ---')
fine_tune_model.summary()

history_finetune = fine_tune_model.fit(
    train_generator,
    epochs=NUM_EPOCHS_FINETUNE,
    validation_data=validation_generator
)

# ---------------------------
# Task 7 & 8: Save loss and accuracy plots for fine-tuned model
# ---------------------------
plt.figure()
plt.plot(history_finetune.history['loss'], label='train_loss')
plt.plot(history_finetune.history['val_loss'], label='val_loss')
plt.title('Fine-tune Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('task7_finetune_loss.png', bbox_inches='tight')
print("Saved 'task7_finetune_loss.png' (Task 7).")

plt.figure()
plt.plot(history_finetune.history['accuracy'], label='train_accuracy')
plt.plot(history_finetune.history['val_accuracy'], label='val_accuracy')
plt.title('Fine-tune Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('task8_finetune_accuracy.png', bbox_inches='tight')
print("Saved 'task8_finetune_accuracy.png' (Task 8).")

# ---------------------------
# Save models
# ---------------------------
extract_model.save('extract_model.h5')
fine_tune_model.save('fine_tune_model.h5')
print('Saved extract_model.h5 and fine_tune_model.h5')

# ---------------------------
# Task 9: Plot a test image using Extract Features Model (index_to_plot = 1)
# ---------------------------
test_generator.reset()

filenames = test_generator.filenames
true_labels = test_generator.classes
class_indices = test_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

print('Test filenames example:', filenames[:5])
print('Index to class mapping:', index_to_class)

extract_preds = extract_model.predict(test_generator, steps=len(test_generator), verbose=1)
extract_pred_classes = np.argmax(extract_preds, axis=1)

idx = INDEX_TO_PLOT
img_path = os.path.join(test_dir, filenames[idx])
img = keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
img_arr = keras.preprocessing.image.img_to_array(img)/255.0

plt.figure()
plt.imshow(img_arr)
plt.axis('off')
pred_label = index_to_class[int(extract_pred_classes[idx])]
true_label = index_to_class[int(true_labels[idx])]
plt.title(f'Extract Model Prediction: {pred_label} | True: {true_label}')
plt.savefig('task9_extract_test_image.png', bbox_inches='tight')
print("Saved 'task9_extract_test_image.png' (Task 9).")

# ---------------------------
# Task 10: Plot a test image using Fine-Tuned Model (index_to_plot = 1)
# ---------------------------
test_generator.reset()
finetune_preds = fine_tune_model.predict(test_generator, steps=len(test_generator), verbose=1)
finetune_pred_classes = np.argmax(finetune_preds, axis=1)

plt.figure()
plt.imshow(img_arr)
plt.axis('off')
pred_label_ft = index_to_class[int(finetune_pred_classes[idx])]
plt.title(f'Fine-tuned Model Prediction: {pred_label_ft} | True: {true_label}')
plt.savefig('task10_finetune_test_image.png', bbox_inches='tight')
print("Saved 'task10_finetune_test_image.png' (Task 10).")

print('\\nAll required artifacts are saved as PNG files in the working directory:')
print('- task6_extract_accuracy.png')
print('- task7_finetune_loss.png')
print('- task8_finetune_accuracy.png')
print('- task9_extract_test_image.png')
print('- task10_finetune_test_image.png')

TensorFlow version: 2.19.0


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/train'